# Infobip
This notebook that shows how to use [Infobip](https://www.infobip.com/) API wrapper to send SMS messages, emails or validate emails.

Infobip provides many services, but this notebook will focus on SMS and email services. You can find more information about the API and other channels [here](https://www.infobip.com/docs/api).

## Setup

To use this tool you need to have an Infobip account. You can create [free trial account](https://www.infobip.com/docs/essentials/free-trial).


`InfobipAPIWrapper` uses name parameters where you can provide credentials:
- `infobip_api_key` - API key that you can find in your [developer tools](https://portal.infobip.com/dev/api-keys)
- `infobip_base_url` - base URL for Infobip API. You can use default value `https://api.infobip.com/`.

You can also provide `infobip_api_key` and `infobip_base_url` as environment variables `INFOBIP_API_KEY` and `INFOBIP_BASE_URL`.

## Sending a SMS

In [ ]:
from langchain_community.utilities.infobip import InfobipAPIWrapper

infobip: InfobipAPIWrapper = InfobipAPIWrapper()

infobip.run(
    to="+821012345678",
    text="Hello, World!",
    sender="Langchain",
    channel="sms",
)

## Sending a email

In [ ]:
from langchain_community.utilities.infobip import InfobipAPIWrapper

infobip: InfobipAPIWrapper = InfobipAPIWrapper()

infobip.run(
    to="test@test.com",
    sender="test@test.com",
    subject="example",
    body="example",
    channel="email",
)

## Validate email

In [ ]:
from langchain_community.utilities.infobip import InfobipAPIWrapper

infobip: InfobipAPIWrapper = InfobipAPIWrapper()

infobip.run(
    to="test@test.com",
    channel="email-validation",
)

# How to use it inside an Agent 

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_openai_functions_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_community.utilities.infobip import InfobipAPIWrapper
from langchain_openai import ChatOpenAI

instructions = """
You are a communication agent. You are talking to a customer who wants to know about the product.
The customer asks you about the product. You answer the customer's question via email if you have a offer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
llm = ChatOpenAI(temperature=0)


class EmailInput(BaseModel):
    body: str = Field(description="Email body text")
    to: int = Field(description="Email address to send to")
    sender: str = Field(description="Email address to send from")
    subject: str = Field(description="Email subject")
    channel: str = Field(
        description="Channel type (email, sms, email-validation) (default: email)"
    )


infobip_api_wrapper: InfobipAPIWrapper = InfobipAPIWrapper()
infobip_tool = Tool(
    name="infobip_sms",
    description="Send SMS via Infobip. If you need to send email, use infobip_email",
    func=infobip_api_wrapper.run,
)
tools = [infobip_tool]

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

agent_executor.invoke(
    {"input": "Which products are usable inside langchain community?"}
)